# Supercut Template
A notebook to copy to make supercuts from. Subject to change based on experience.

In [45]:
from esper.supercuts import *

In [46]:
def query():
    '''
    Write your query here. Below is a query for Harry, Ron, and Hermione in a row.
    This function should return a rekall VideoIntervalCollection.
    '''
    from esper.captions import topic_search
    
    phrases = [
        'may the Force be with you',
        'may the force be with you',
        'May the Force be with you',
        'May the force be with you'
    ]
    results = topic_search(phrases,
        window_size=0,
        video_ids = [vid.id for vid in Video.objects.filter(name__contains="star wars").all()])
    caption_results = VideoIntervalCollection({
        r.id: [(
            (p.start * Video.objects.get(id=r.id).fps),
            (p.end * Video.objects.get(id=r.id).fps),
        0) for p in r.postings]
        for r in results
    })
    
    return caption_results

In [ ]:
def generate_supercut_intervals(
    interval_collection,
    dilation_start=0,
    dilation_end=0,
    dilation=0,
    snap_to_cinematic_shots=False,
    limit=None,
    stride=1
):
    '''
    Generates supercut intervals for the supercut pipeline.
    Dilates interval_collection by dilation.
    If dilation > 0, snaps the dilations to cinematic shot boundaries.
    '''
    if dilation > 0:
        if snap_to_cinematic_shots:
            shots = VideoIntervalCollection.from_django_qs(
                Shot.objects.filter(
                    video_id__in=list(interval_collection.get_allintervals().keys()),
                    labeler__name="shot-hsvhist-face"
                )
            ).filter_against(interval_collection, predicate=overlaps())
        interval_collection = interval_collection.dilate(dilation).coalesce()
        if snap_to_cinematic_shots:
            interval_collection = interval_collection.overlaps(shots)
    
    supercut_intervals = []
    for video_id in sorted(list(interval_collection.get_allintervals().keys())):
        intervallist = interval_collection.get_intervallist(video_id)
        for intrvl in intervallist.get_intervals():
            supercut_intervals.append((video_id, intrvl.get_start()-dilation_start, intrvl.get_end()+dilation_end))
            
    if limit is not None:
        return supercut_intervals[:limit * stride:stride]
    
    return supercut_intervals[::stride]

In [ ]:
# output path
OUTPUT_PATH = '/app/result/supercut/may_the_force_be_with_you.mp4'

In [ ]:
stitch_video_temporal(generate_supercut_intervals(
    query(), dilation_end=12), out_path=OUTPUT_PATH, width=640, height=480)